In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver

options = webdriver.ChromeOptions()
options.add_argument('window-size=1920x1080')
options.add_experimental_option("excludeSwitches", ["enable-logging"])
driver = webdriver.Chrome(options=options)

In [2]:
import time

driver.get('https://www.abf.gov.au/help-and-support/notices/australian-customs-notices')
time.sleep(2)

In [3]:
soup = BeautifulSoup(driver.page_source, 'lxml')

In [4]:
temp = soup.find('ul', attrs={'class' : 'pagination'}).find_all("li")

In [5]:
from selenium.webdriver.common.by import By
import re

In [6]:
dates = []
titles = []
links = []
last_page = len(temp)

for i in range(2,last_page):
    soup = BeautifulSoup(driver.page_source, 'lxml')
    
    date_rows = soup.select('div.col-sm-3 span')
    title_rows = soup.find('tbody').find_all('tr', attrs = {'class' : re.compile('^accordion-header')})
    link_rows = soup.find('tbody').find_all('tr', attrs = {'class' : re.compile('^accordion-content')})
    
    for row in date_rows:
        date = row.text
        dates.append(date)

    for row in title_rows:
        tds = row.find_all('td')
        title = tds[1].get_text().replace('Press Enter to show more details.', '')  
        titles.append(title)

    for row in link_rows:
        link = 'https://www.abf.gov.au' + row.a['href']
        links.append(link)
    
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
    time.sleep(2)
    
    elem = driver.find_element(By.LINK_TEXT,str(i))
    elem.click()
    time.sleep(1)

In [7]:
print(len(dates))
print(len(titles))
print(len(links))

1680
1680
1680


In [8]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.layout import LAParams
from pdfminer.converter import TextConverter
from pdfminer.pdfpage import PDFPage
import io
import urllib.request
import requests


def pdf_to_text(pdf_file):
    text_memory_file = io.StringIO()

    rsrcmgr = PDFResourceManager()
    device = TextConverter(rsrcmgr, text_memory_file, laparams=LAParams())
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    
    # get first 3 pages of the pdf file
    for page in PDFPage.get_pages(pdf_file):
        interpreter.process_page(page)
    text = text_memory_file.getvalue()
    text_memory_file.close()
    return text



In [32]:
# texts = []
start = 751
finish = len(dates)

for i in range(start, finish): 
    split_texts = []
    
    url = links[i]
    response = requests.get(url)
    my_raw_data = response.content
    
    # online pdf to text by requests
    response = requests.get(url)
    pdf_memory_file = io.BytesIO()
    pdf_memory_file.write(response.content)
    
    pdf_text = pdf_to_text(pdf_memory_file)
    pdf_text = pdf_text.lower()

    #[signed] 찾기 위해 단어 단위로 쪼개기
    split_words = pdf_text.split()
    
    sentence = ""
    for word in range(len(split_words)):
        
        #[signed]가 있는 경우
        if split_words[word] == "[signed]":
            texts.append(sentence)
            break
                  
        #assistant secretary가 있는 경우
        elif split_words[word] == "assistant" and len(split_words) > word+1:
            if split_words[word+1] == "secretary":
                texts.append(sentence)
                break
            else:
                sentence = sentence + " " + split_words[word]
                
        #customs and trade policy branch가 있는 경우
        elif split_words[word]== "customs" and len(split_words) > word+4:
            if split_words[word+1]=="and" and split_words[word+2]=="trade" and split_words[word+3]=="policy" and split_words[word+4]=="branch":
                texts.append(sentence)
                break
            else:
                sentence = sentence + " " + split_words[word]
             
        elif len(split_words) == word+1:
            sentence = sentence + " " + split_words[word]
            texts.append(sentence)

        
        else:
            sentence = sentence + " " + split_words[word]
        
       
    if len(texts)<i+1:
        texts.append("NONE")
        print(i+1, "번 문서 종료 / 총 길이는 : ", len(texts[i]))
        print(texts[i][:80])
    else:
        print(i+1, "번 문서 종료 / 총 길이는 : ", len(texts[i]))
        print(texts[i][:80])


752 번 문서 종료 / 총 길이는 :  319738
 a u s t r a l i a n c u s t o m s n o t i c e n o . 2 0 0 8 / 1 4 notices of ob
753 번 문서 종료 / 총 길이는 :  5057
 australian customs notice no. 2008/13 import and export controls for un-sanctio
754 번 문서 종료 / 총 길이는 :  1259
 a u s t r a l i a n c u s t o m s n o t i c e n o . 2 0 0 8 / 1 2 application f
755 번 문서 종료 / 총 길이는 :  1541
 a u s t r a l i a n c u s t o m s n o t i c e n o . 2 0 0 8 / 0 8 application f
756 번 문서 종료 / 총 길이는 :  2382
 australian customs notice no. 2008/07 control permit for non-commercial foreign
757 번 문서 종료 / 총 길이는 :  19631
 australian customs notice no. 2008/06 indexation of certain customs and excise 
758 번 문서 종료 / 총 길이는 :  1661
 a u s t r a l i a n c u s t o m s n o t i c e n o . 2 0 0 8 / 0 5 application f
759 번 문서 종료 / 총 길이는 :  2090
 australian customs notice no. 2008/04 revocation of customs notice no. 1 (2007)
760 번 문서 종료 / 총 길이는 :  2145
 a u s t r a l i a n c u s t o m s n o t i c e n o . 2 0 0 8 / 0 3 application f
761 번 문서 종료 / 총 

827 번 문서 종료 / 총 길이는 :  1995
 australian customs notice no. 2006/64 new rules of origin – australia new zeala
828 번 문서 종료 / 총 길이는 :  2380
 australian customs notice no. 2006/62 singapore - australia free trade agreemen
829 번 문서 종료 / 총 길이는 :  340473
 a u s t r a l i a n c u s t o m s n o t i c e n o . 2 0 0 6 / 6 1 notices of ob
830 번 문서 종료 / 총 길이는 :  759
 a u s t r a l i a n c u s t o m s n o t i c e n o . 2 0 0 6 / 6 0 application f
831 번 문서 종료 / 총 길이는 :  2177
 australian customs notice no. 2006/59 definition of "consignment" for the purpo
832 번 문서 종료 / 총 길이는 :  2576
 australian customs notice no. 2006/58 australia–united states free trade agreem
833 번 문서 종료 / 총 길이는 :  2324
 australian customs notice no. 2006/57 tariff working pages and tariff advices –
834 번 문서 종료 / 총 길이는 :  1651
 a u s t r a l i a n c u s t o m s n o t i c e n o . 2 0 0 6 / 5 6 united nation
835 번 문서 종료 / 총 길이는 :  1121
 a u s t r a l i a n c u s t o m s n o t i c e n o . 2 0 0 6 / 5 5 application f
836 번 문서 종료 / 총 길이

903 번 문서 종료 / 총 길이는 :  1298
 a u s t r a l i a n c u s t o m s n o t i c e n o . 2 0 0 5 / 5 9 application f
904 번 문서 종료 / 총 길이는 :  1522
 australian customs notice no. 2005/58 item 47 to schedule 4 – removal of 3% cus
905 번 문서 종료 / 총 길이는 :  2234
 australian customs notice no. 2005/57 formal import entry and revenue collectio
906 번 문서 종료 / 총 길이는 :  1150
 a u s t r a l i a n c u s t o m s n o t i c e n o . 2 0 0 5 / 5 5 application f
907 번 문서 종료 / 총 길이는 :  3870
 australian customs notice no. 2005/54 australia – united states free trade agre
908 번 문서 종료 / 총 길이는 :  986
 a u s t r a l i a n c u s t o m s n o t i c e n o . 2 0 0 5 / 5 3 application f
909 번 문서 종료 / 총 길이는 :  2451
 australian customs notice no. 2005/52 authorized agents - declaration of places
910 번 문서 종료 / 총 길이는 :  3892
 australian customs notice no. 2005/51 integrated cargo system - changes to admi
911 번 문서 종료 / 총 길이는 :  1969
 australian customs notice no. 2005/50 amendments to the customs (prohibited imp
912 번 문서 종료 / 총 길이는 

979 번 문서 종료 / 총 길이는 :  2076
 australian customs notice no. 2004/37 new exports regulations the purpose of th
980 번 문서 종료 / 총 길이는 :  998
 au s t r a l i a n c u s to m s n ot i c e n o. 2 0 0 4 / 3 6 application for c
981 번 문서 종료 / 총 길이는 :  6180
 australian customs notice no. 2004/34 amendments to the customs (prohibited imp
982 번 문서 종료 / 총 길이는 :  12486
 australian customs notice no. 2004/33 sea cargo examination arrangements this a
983 번 문서 종료 / 총 길이는 :  11313
 australian customs notice no. 2004/32 indexation of certain excise and customs 
984 번 문서 종료 / 총 길이는 :  1918
 australian customs notice no. 2004/31 amendments to the customs (prohibited imp
985 번 문서 종료 / 총 길이는 :  1367
 a u s t r a l i a n c u s t o m s n o t i c e n o . 2 0 0 4 / 2 9 application f
986 번 문서 종료 / 총 길이는 :  1598
 australian customs notice no. 2004/28 no updates to the australian harmonized e
987 번 문서 종료 / 총 길이는 :  1683
 australian customs notice no. 2004/27 customs tariff amendment (fuels) act 2004
988 번 문서 종료 / 총 길이

1054 번 문서 종료 / 총 길이는 :  2074
 australian customs notice no. 2003/34 application for customs broker’s licence 
1055 번 문서 종료 / 총 길이는 :  10908
 australian customs notice no. 2003/33 notice of objection - section 135 of the 
1056 번 문서 종료 / 총 길이는 :  134562
 australian customs notice no. 2003/32 notices of objection to importation under
1057 번 문서 종료 / 총 길이는 :  1666
 australian customs notice no. 2003/31 application for customs broker’s licence 
1058 번 문서 종료 / 총 길이는 :  1519
 australian customs notice no. 2003/30 application for customs broker’s licence 
1059 번 문서 종료 / 총 길이는 :  2089
 australian customs notice no. 2003/29 amendments to the customs (prohibited imp
1060 번 문서 종료 / 총 길이는 :  4086
 australian customs notice no. 2003/28 copyright amendment (parallel importation
1061 번 문서 종료 / 총 길이는 :  5255
 australian customs notice no. 2003/27 _________________________________________
1062 번 문서 종료 / 총 길이는 :  1488
 australian customs notice no. 2003/26 new import control - candles with lead co
1063 번 

...

1354 번 문서 종료 / 총 길이는 :  3086
 australian customs notice 1999 61a notice of objection section 135 of the copyr
1355 번 문서 종료 / 총 길이는 :  1405
 australian customs notice 1999 61 commerce prohibitions and restrictions attach
1356 번 문서 종료 / 총 길이는 :  688
 australian customs notice 1999 60 application for corporate customs brokers lic
1357 번 문서 종료 / 총 길이는 :  1187
 australian customs notice 1999 59 application for corporate customs brokers lic
1358 번 문서 종료 / 총 길이는 :  2413
 australian customs notice 1999 58 government administrative charge - reimbursem
1359 번 문서 종료 / 총 길이는 :  1056
 australian customs notice 1999 57a attachment to 99/57 1. aluminium sheet class
1360 번 문서 종료 / 총 길이는 :  1276
 australian customs notice 1999 57 tariff changes to aluminium cansheet and stee
1361 번 문서 종료 / 총 길이는 :  646
 australian customs notice 1999 56 application for corporate customs brokers lic
1362 번 문서 종료 / 총 길이는 :  600
 australian customs notice 1999 55 application for customs brokers licence the f
1363 번 문서 종료 

1429 번 문서 종료 / 총 길이는 :  3655
 australian customs notice 1999 notices of objection to importation the companie
1430 번 문서 종료 / 총 길이는 :  1399
 australian customs notice 1998 90 customs year 2000 it external testing in sept
1431 번 문서 종료 / 총 길이는 :  578
 australian customs notice 1998 89 application for customs brokers licence the f
1432 번 문서 종료 / 총 길이는 :  3070
 australian customs notice 1998 88 introduction of the euro from 1 january 1999,
1433 번 문서 종료 / 총 길이는 :  3521
 australian customs notice 1998 87 customs tariff proposals nos. 6 to 10 (1998) 
1434 번 문서 종료 / 총 길이는 :  3334
 australian customs notice 1998 86 notices of objection to importation trade mar
1435 번 문서 종료 / 총 길이는 :  1949
 australian customs notice 1998 85 notice of objection section 135 of the copyri
1436 번 문서 종료 / 총 길이는 :  638
 australian customs notice 1998 84 application for customs brokers licence the f
1437 번 문서 종료 / 총 길이는 :  755
 australian customs notice 1998 83 application for customs brokers licence the f
1438 번 문서 종료 

1504 번 문서 종료 / 총 길이는 :  2093
 australian customs notice 1998 35 sales tax accreditation for computer goods th
1505 번 문서 종료 / 총 길이는 :  1415
 australian customs notice 1998 34 b attachment b - sales tax exemption declarat
1506 번 문서 종료 / 총 길이는 :  7493
 australian customs notice 1998 34 a attachment a - sales tax exemption item num
1507 번 문서 종료 / 총 길이는 :  2814
 australian customs notice 1998 34 sales tax exemption changes the australian ta
1508 번 문서 종료 / 총 길이는 :  5306
 australian customs notice 1998 33 notices of objection - section 135 of the cop
1509 번 문서 종료 / 총 길이는 :  9046
 australian customs notice 1998 32 s schedule to acn 98/32 trade mark notices of
1510 번 문서 종료 / 총 길이는 :  1384
 australian customs notice 1998 32 c commerce prohibitions and restrictions regi
1511 번 문서 종료 / 총 길이는 :  6865
 australian customs notice 1998 32 1 anti-dumping and countervailing measures - 
1512 번 문서 종료 / 총 길이는 :  3395
 australian customs notice 1998 32 notices of objection to importation - trade m
1513 번 문서 

1579 번 문서 종료 / 총 길이는 :  2600
 australian customs notice 1997 52 revocation of certain item 47 by-laws a recen
1580 번 문서 종료 / 총 길이는 :  2624
 australian customs notice 1997 51 customs tariff proposal no. 4 (1997) miscella
1581 번 문서 종료 / 총 길이는 :  13917
 australian customs notice 1997 50 information technology agreement customs tari
1582 번 문서 종료 / 총 길이는 :  10575
 australian customs notice 1997 49 policy by-law system required format of items
1583 번 문서 종료 / 총 길이는 :  2938
 australian customs notice 1997 48 renewal of warehouse licences and payment of 
1584 번 문서 종료 / 총 길이는 :  1152
 australian customs notice 1997 47 21 customs act 1901 - part xvb withdrawal of 
1585 번 문서 종료 / 총 길이는 :  10004
 australian customs notice 1997 47 2 notice of objection to importation trade ma
1586 번 문서 종료 / 총 길이는 :  1743
 australian customs notice 1997 46 tariff quotas - cheese and curd allocation fo
1587 번 문서 종료 / 총 길이는 :  2854
 australian customs notice 1997 44 1997/1998 budget changes affecting sales tax 
1588 번 

1654 번 문서 종료 / 총 길이는 :  8885
 australian customs notice 1996 34 indexation of certain excise and customs tari
1655 번 문서 종료 / 총 길이는 :  4305
 australian customs notice 1996 33 eligibility for bounty of computer operating 
1656 번 문서 종료 / 총 길이는 :  52440
 australian customs notice 1996 32 the policy by-law system (items 43, 45, 46, 4
1657 번 문서 종료 / 총 길이는 :  4361
 australian customs notice 1996 31 amendments to the administrative arrangements
1658 번 문서 종료 / 총 길이는 :  16509
 australian customs notice 1996 28 amendments to the tariff concession system in
1659 번 문서 종료 / 총 길이는 :  2548
 australian customs notice 1996 27 australian customs service publications price
1660 번 문서 종료 / 총 길이는 :  1737
 australian customs notice 1996 26 tariff quotas - cheese and curd allocation fo
1661 번 문서 종료 / 총 길이는 :  7485
 australian customs notice 1996 25 amendments to customs (prohibited imports) re
1662 번 문서 종료 / 총 길이는 :  2868
 australian customs notice 1996 24 re-instatement of export controls on schedule
1663 번 문

In [26]:
texts.append("NONE")

In [30]:
print(len(texts))

371
371 번 문서 종료 / 총 길이는 :  4
NONE


In [33]:
import pandas as pd

df = pd.DataFrame()
df['date'] = pd.to_datetime(dates)
df['link'] = links
df['title'] = titles
df['text'] = texts

df

,date,link,title,text
0,2022-07-28,https://www.abf.gov.au/help-and-support-subsit...,Indexation of customs duty rates on excise-equ...,australian customs notice no. 2022/35 indexat...
1,2022-07-22,https://www.abf.gov.au/help-and-support-subsit...,Removal of customs duty on certain electric ve...,australian customs notice no. 2022/34 removal...
2,2022-07-21,https://www.abf.gov.au/help-and-support-subsit...,Application for Customs Broker Licences,australian customs notice no. 2022/33 applica...
3,2022-07-12,https://www.abf.gov.au/help-and-support-subsit...,Temporary duty reduction for goods from Ukraine,australian customs notice no. 2022/32 tempora...
4,2022-07-11,https://www.abf.gov.au/help-and-support-subsit...,Application for Customs Broker Licences,australian customs notice no. 2022/31 applica...
...,...,...,...,...
1675,2018-12-10,https://www.abf.gov.au/help-and-support-subsit...,Preparation of submanifests for export cargo c...,australian customs notice 1996 09 preparation...
1676,2018-12-10,https://www.abf.gov.au/help-and-support-subsit...,Section 135 of the Copyright Act 1968,australian customs notice 1996 07 section 135...
1677,2018-12-10,https://www.abf.gov.au/help-and-support-subsit...,Sea cargo automation: electronic transhipment ...,australian customs notice 1996 06 sea cargo a...
1678,2018-12-10,https://www.abf.gov.au/help-and-support-subsit...,Indexation of certain excise and customs tarif...,australian customs notice 1996 05 29 indexati...
